In [1]:
import torch
from torch.nn import CrossEntropyLoss
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
import random
import nltk 
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from dotenv import load_dotenv
from nltk.corpus import wordnet

from degree_inference.cah_data import CAHData
from degree_inference.predict import predict
from degree_inference.train import train

load_dotenv() 

from google.cloud import bigquery

%load_ext google.cloud.bigquery

In [2]:
%%bigquery degree_name_to_hecos
SELECT
  LOWER(cq.subject) AS text,
  cq.degree_subject_cah_l3 as label
FROM
  `rugged-abacus-218110.dataform_ABS_2_dev.application_choice_details`
LEFT JOIN
  UNNEST(candidate_qualifications) AS cq
LEFT JOIN `rugged-abacus-218110.dfe_reference_data.cah_categories_l3_v2` AS cah_codes ON cah_codes.id = degree_subject_cah_l3
WHERE degree_level IS NOT NULL AND degree_level !='unknown'
AND degree_subject_cah_l3 IS NOT NULL
GROUP BY 
text,
label

Query is running:   0%|          |

Downloading:   0%|          |

In [3]:
data = CAHData(augment=True)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data.df['label'].unique()))
mps_device = torch.device("mps")
model.to(mps_device)

trainer = train(model,data,epochs=30)
model.to("cpu")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/duncan/projects/transformers/.venv/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
out = predict(data, model,list(degree_name_to_hecos['text']))
out

In [ ]:
# model.save_pretrained("./30-epoch-gpt2-ilr-augmented-1e-5")